<a href="https://colab.research.google.com/github/elka97/pythonLibrariesCourse/blob/main/baseline_RealEstateMoscowCourse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Подключение библиотек и скриптов

In [ ]:
import numpy as np
import pandas as pd
import random
from scipy.stats import norm
from scipy import stats

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.ensemble import RandomForestRegressor, StackingRegressor, VotingRegressor, BaggingRegressor, GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

from sklearn.metrics import r2_score as r2
from sklearn.model_selection import KFold, GridSearchCV

from datetime import datetime

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

matplotlib.rcParams.update({'font.size': 12})

In [ ]:
    print("Train R2:\t" + str(round(r2(train_true_values, train_pred_values), 3)))
    print("Test R2:\t" + str(round(r2(test_true_values, test_pred_values), 3)))
    
    plt.figure(figsize=(18,10))
    
    plt.subplot(121)
    sns.scatterplot(x=train_pred_values, y=train_true_values)
    plt.xlabel('Predicted values')
    plt.ylabel('True values')
    plt.title('Train sample prediction')
    
    plt.subplot(122)
    sns.scatterplot(x=test_pred_values, y=test_true_values)
    plt.xlabel('Predicted values')
    plt.ylabel('True values')
    plt.title('Test sample prediction')

    plt.show()

Пути к директориям и файлам

In [ ]:
TRAIN_DATASET_PATH = '/kaggle/input/real-estate-price-prediction-moscow/train.csv'
TEST_DATASET_PATH = '/kaggle/input/real-estate-price-prediction-moscow/test.csv'

TRAIN_DATASET_PATH = '/kaggle/input/real-estate-price-prediction-moscow/train.csv'
TEST_DATASET_PATH = '/kaggle/input/real-estate-price-prediction-moscow/test.csv'


Загрузка данных
Описание датасета

Id - идентификационный номер квартиры
DistrictId - идентификационный номер района
Rooms - количество комнат
Square - площадь
LifeSquare - жилая площадь
KitchenSquare - площадь кухни
Floor - этаж
HouseFloor - количество этажей в доме
HouseYear - год постройки дома
Ecology_1, Ecology_2, Ecology_3 - экологические показатели местности
Social_1, Social_2, Social_3 - социальные показатели местности
Healthcare_1, Helthcare_2 - показатели местности, связанные с охраной здоровья
Shops_1, Shops_2 - показатели, связанные с наличием магазинов, торговых центров
Price - цена квартиры

In [ ]:
train_df = pd.read_csv(TRAIN_DATASET_PATH)
train_df.tail()

In [ ]:
train_df.dtypes

In [ ]:
test_df = pd.read_csv(TEST_DATASET_PATH)
test_df.tail()

In [ ]:
print('Строк в трейне:', train_df.shape[0])
print('Строк в тесте', test_df.shape[0])

In [ ]:
train_df.shape[1] - 1 == test_df.shape[1]

In [ ]:
Приведение типов

In [ ]:
train_df.dtypes

In [ ]:
train_df['Id'] = train_df['Id'].astype(str)
train_df['DistrictId'] = train_df['DistrictId'].astype(str)

**# 1. EDA**

Делаем EDA для:
1.  Исправления выбросов 
2.  Заполнения NaN 
3.  Идей для генерации новых фич

Целевая переменная

In [ ]:
plt.figure(figsize = (16, 8))

train_df['Price'].hist(bins=30)
plt.ylabel('Count')
plt.xlabel('Price')

plt.title('Target distribution')
plt.show()

Количественные переменные

In [ ]:
train_df.describe()

Номинативные переменные

In [ ]:
train_df.select_dtypes(include='object').columns.tolist()

In [ ]:
train_df['DistrictId'].value_counts()

In [ ]:
train_df['Ecology_2'].value_counts()

In [ ]:
train_df['Ecology_3'].value_counts()

In [ ]:
train_df['Shops_2'].value_counts()

**2. Обработка выбросов**
Что можно делать с ними?

Выкинуть эти данные (только на трейне, на тесте ничего не выкидываем)
Заменять выбросы разными методами (медианы, средние значения, np.clip и т.д.)
Делать/не делать дополнительную фичу
Ничего не делать

Rooms

In [ ]:
train_df['Rooms'].value_counts()

In [ ]:
train_df['Rooms_outlier'] = 0
train_df.loc[(train_df['Rooms'] == 0) | (train_df['Rooms'] >= 6), 'Rooms_outlier'] = 1
train_df.head()

In [ ]:
train_df.loc[train_df['Rooms'] == 0, 'Rooms'] = 1
train_df.loc[train_df['Rooms'] >= 6, 'Rooms'] = train_df['Rooms'].median()
#train_df.loc[train_df['Rooms'] >= 6, 'Rooms'] = train_df['Rooms'].quantile(.975)
#попробовать заменить на train_df['Rooms'].quantile(.975) для  >= 6 

In [ ]:
train_df['Rooms'].value_counts()

KitchenSquare

In [ ]:
train_df['KitchenSquare'].value_counts()

In [ ]:
train_df['KitchenSquare'].quantile(.975), train_df['KitchenSquare'].quantile(.025)

In [ ]:
condition = (train_df['KitchenSquare'].isna()) \
             | (train_df['KitchenSquare'] > train_df['KitchenSquare'].quantile(.975))

train_df['KitchenSquare_outlier'] = 0
train_df.loc[condition | (train_df['KitchenSquare'] < 3), 'KitchenSquare_outlier'] = 1

train_df.loc[condition, 'KitchenSquare'] = train_df['KitchenSquare'].median()
train_df.loc[train_df['KitchenSquare'] < 3, 'KitchenSquare'] = 3
# кухни, размер которых больше разницы между общей и жилой площадью - заменяем на разницу между общей и жилой площадью
kitchenSquareCondition = (train_df['KitchenSquare'] > train_df['Square'] - train_df['LifeSquare']) & (train_df['Square'] > train_df['LifeSquare']) & (~train_df['LifeSquare'].isna())
train_df.loc[kitchenSquareCondition, 'KitchenSquare_outlier'] = 1
train_df.loc[kitchenSquareCondition, 'KitchenSquare'] = train_df['Square'] - train_df['LifeSquare']
train_df['KitchenSquare'].value_counts()

In [ ]:
train_df['KitchenSquare'].value_counts()

HouseFloor, Floor

In [ ]:
train_df['HouseFloor'].sort_values()

In [ ]:
train_df['Floor'].sort_values().unique()

In [ ]:
(train_df['Floor'] > train_df['HouseFloor']).sum()

In [ ]:
train_df['HouseFloor_outlier'] = 0
train_df.loc[train_df['HouseFloor'] == 0, 'HouseFloor_outlier'] = 1
train_df.loc[train_df['Floor'] > train_df['HouseFloor'], 'HouseFloor_outlier'] = 1
train_df.loc[train_df['HouseFloor'] > train_df['HouseFloor'].quantile(.975), 'HouseFloor_outlier'] = 1

In [ ]:
train_df.loc[train_df['HouseFloor'] > train_df['HouseFloor'].quantile(.975), 'HouseFloor'] = train_df['HouseFloor'].quantile(.975)
train_df.loc[train_df['HouseFloor'] == 0, 'HouseFloor'] = train_df['HouseFloor'].median()

In [ ]:
floor_outliers = train_df.loc[train_df['Floor'] > train_df['HouseFloor']].index
floor_outliers

In [ ]:
train_df.loc[floor_outliers, 'Floor'] = train_df.loc[floor_outliers, 'HouseFloor']\
                                                .apply(lambda x: random.randint(1, x))

In [ ]:
(train_df['Floor'] > train_df['HouseFloor']).sum()

HouseYear

In [ ]:
train_df['HouseYear'].sort_values(ascending=False)

In [ ]:
train_df.loc[train_df['HouseYear'] > 2021, 'HouseYear'] = 2021

3. Обработка пропусков

In [ ]:
train_df.isna().sum()

In [ ]:
train_df[['Square', 'LifeSquare', 'KitchenSquare']].head(10)

LifeSquare

In [ ]:
train_df['LifeSquare_nan'] = train_df['LifeSquare'].isna() * 1

condition = (train_df['LifeSquare'].isna()) \
             & (~train_df['Square'].isna()) \
             & (~train_df['KitchenSquare'].isna())
        
train_df.loc[condition, 'LifeSquare'] = train_df.loc[condition, 'Square'] \
                                            - train_df.loc[condition, 'KitchenSquare'] - 3

Healthcare_1

In [ ]:
train_df.drop('Healthcare_1', axis=1, inplace=True)

In [ ]:
class DataPreprocessing:
    """Подготовка исходных данных"""

    def __init__(self):
        """Параметры класса"""
        self.medians=None
        self.kitchen_square_quantile = None
        self.house_floor_quantile = None

    def fit(self, X):
        """Сохранение статистик"""       
        # Расчет медиан
        self.medians = X.median()
        self.kitchen_square_quantile = X['KitchenSquare'].quantile(.975)
        self.house_floor_quantile = X['HouseFloor'].quantile(.975)
    
    def transform(self, X):
        """Трансформация данных"""

        # Rooms
        X['Rooms_outlier'] = 0
        X.loc[(X['Rooms'] == 0) | (X['Rooms'] >= 6), 'Rooms_outlier'] = 1
        
        ###X.loc[X['Rooms'] == 0, 'Rooms'] = 1
        ###X.loc[X['Rooms'] >= 6, 'Rooms'] = self.medians['Rooms']
        
        info_by_district_id = X.groupby(['DistrictId', 'HouseYear'], as_index=False).agg({'Rooms': 'sum', 'Square': 'sum'}).rename(columns={'Rooms': 'sum_rooms_dr', 'Square': 'sum_square_dr'})

        info_by_district_id['mean_square_per_room_in_dr'] = info_by_district_id['sum_square_dr'] / info_by_district_id['sum_rooms_dr']
        info_by_district_id.drop(['sum_square_dr', 'sum_rooms_dr'], axis=1, inplace=True)

        X = pd.merge(X, info_by_district_id, on=['DistrictId', 'HouseYear'], how='left')

        X['mean_square_per_room_in_dr'] = X['mean_square_per_room_in_dr'].fillna(X['mean_square_per_room_in_dr'].mean())

        X.loc[X['Rooms'] >= 6, 'Rooms'] = (X.loc[X['Rooms'] > 6, 'Square'] // X.loc[X['Rooms'] > 6, 'mean_square_per_room_in_dr']).astype('int')

        X.loc[X['Rooms'] == 0, 'Rooms'] = (X.loc[X['Rooms'] == 0, 'Square'] // X.loc[X['Rooms'] == 0, 'mean_square_per_room_in_dr']).astype('int')        
        
        
        # KitchenSquare
        condition = (X['KitchenSquare'].isna()) \
                    | (X['KitchenSquare'] > self.kitchen_square_quantile)
        
        X.loc[condition, 'KitchenSquare'] = self.medians['KitchenSquare']

        X.loc[X['KitchenSquare'] < 3, 'KitchenSquare'] = 3
        
        # Square
        X.loc[X['Square'] > 400, 'Square'] = X.loc[X['Square'] > 400, 'Square'] / 10
        
        mu, sigma = norm.fit(X['Square'])

        info_by_district_id = X.groupby(['DistrictId', 'Rooms', 'HouseYear'], as_index=False).agg({'Square': 'mean'}).rename(columns={'Square': 'mean_square_rooms_dr'})

        X = pd.merge(X, info_by_district_id, on=['DistrictId', 'Rooms', 'HouseYear'], how='left')

        X.loc[abs(X['Square'] - X['mean_square_rooms_dr']) > 2 * sigma, 'Square'] \
        = X.loc[abs(X['Square'] - X['mean_square_rooms_dr']) > 2 * sigma, 'Rooms'] \
        * X.loc[abs(X['Square'] - X['mean_square_rooms_dr']) > 2 * sigma, 'mean_square_per_room_in_dr']
       
        # кухни, размер которых больше разницы между общей и жилой площадью - заменяем на разницу между общей и жилой площадью
        kitchenSquareCondition = (X['KitchenSquare'] > X['Square'] - X['LifeSquare']) & (X['Square'] > X['LifeSquare']) & (~X['LifeSquare'].isna())
        X.loc[kitchenSquareCondition, 'KitchenSquare_outlier'] = 1
        X.loc[kitchenSquareCondition, 'KitchenSquare'] = train_df['Square'] - train_df['LifeSquare']
        
        # HouseFloor, Floor
        X['HouseFloor_outlier'] = 0
        X.loc[X['HouseFloor'] == 0, 'HouseFloor_outlier'] = 1
        X.loc[X['HouseFloor'] > self.house_floor_quantile, 'HouseFloor_outlier'] = 1
        X.loc[X['Floor'] > X['HouseFloor'], 'HouseFloor_outlier'] = 1        
        
        X.loc[X['HouseFloor'] == 0, 'HouseFloor'] = self.medians['HouseFloor']
        X.loc[X['HouseFloor'] > self.house_floor_quantile, 'HouseFloor'] = self.house_floor_quantile
        
        floor_outliers = X.loc[X['Floor'] > X['HouseFloor']].index
        X.loc[floor_outliers, 'Floor'] = X.loc[floor_outliers, 'HouseFloor']\
                                            .apply(lambda x: random.randint(1, x))       

        # HouseYear
        current_year = datetime.now().year
        
        X['HouseYear_outlier'] = 0
        X.loc[X['HouseYear'] > current_year, 'HouseYear_outlier'] = 1
        
        X.loc[X['HouseYear'] > current_year, 'HouseYear'] = current_year
        
        # Healthcare_1
        if 'Healthcare_1' in X.columns:
            X.drop('Healthcare_1', axis=1, inplace=True)
            
        # LifeSquare
        X['LifeSquare_nan'] = X['LifeSquare'].isna() * 1
        condition = (X['LifeSquare'].isna()) & \
                      (~X['Square'].isna()) & \
                      (~X['KitchenSquare'].isna())
        
        X.loc[condition, 'LifeSquare'] = X.loc[condition, 'Square'] - X.loc[condition, 'KitchenSquare'] - 3
        
        
        X.fillna(self.medians, inplace=True)
        
        return X

4. Построение новых признаков

In [ ]:
binary_to_numbers = {'A': 0, 'B': 1}

train_df['Ecology_2'] = train_df['Ecology_2'].replace(binary_to_numbers)
train_df['Ecology_3'] = train_df['Ecology_3'].replace(binary_to_numbers)
train_df['Shops_2'] = train_df['Shops_2'].replace(binary_to_numbers)

DistrictSize, IsDistrictLarge

In [ ]:
district_size = train_df['DistrictId'].value_counts().reset_index()\
                    .rename(columns={'index':'DistrictId', 'DistrictId':'DistrictSize'})

district_size.head()

In [ ]:
train_df = train_df.merge(district_size, on='DistrictId', how='left')
train_df.head()

In [ ]:
(train_df['DistrictSize'] > 100).value_counts()

In [ ]:
train_df['IsDistrictLarge'] = (train_df['DistrictSize'] > 100).astype(int)

MedPriceByDistrict

In [ ]:
med_price_by_district = train_df.groupby(['DistrictId', 'Rooms'], as_index=False).agg({'Price':'median'})\
                            .rename(columns={'Price':'MedPriceByDistrict'})

med_price_by_district.head()

In [ ]:
train_df = train_df.merge(med_price_by_district, on=['DistrictId', 'Rooms'], how='left')
train_df.head()

MedPriceByFloorYear

In [ ]:
def floor_to_cat(X):

    X['floor_cat'] = 0

    X.loc[X['Floor'] <= 3, 'floor_cat'] = 1  
    X.loc[(X['Floor'] > 3) & (X['Floor'] <= 5), 'floor_cat'] = 2
    X.loc[(X['Floor'] > 5) & (X['Floor'] <= 9), 'floor_cat'] = 3
    X.loc[(X['Floor'] > 9) & (X['Floor'] <= 15), 'floor_cat'] = 4
    X.loc[X['Floor'] > 15, 'floor_cat'] = 5

    return X


def floor_to_cat_pandas(X):
    bins = [0, 3, 5, 9, 15, X['Floor'].max()]
    X['floor_cat'] = pd.cut(X['Floor'], bins=bins, labels=False)
    
    X['floor_cat'].fillna(-1, inplace=True)
    return X


def year_to_cat(X):

    X['year_cat'] = 0

    X.loc[X['HouseYear'] <= 1941, 'year_cat'] = 1
    X.loc[(X['HouseYear'] > 1941) & (X['HouseYear'] <= 1945), 'year_cat'] = 2
    X.loc[(X['HouseYear'] > 1945) & (X['HouseYear'] <= 1980), 'year_cat'] = 3
    X.loc[(X['HouseYear'] > 1980) & (X['HouseYear'] <= 2000), 'year_cat'] = 4
    X.loc[(X['HouseYear'] > 2000) & (X['HouseYear'] <= 2010), 'year_cat'] = 5
    X.loc[(X['HouseYear'] > 2010), 'year_cat'] = 6

    return X


def year_to_cat_pandas(X):
    bins = [0, 1941, 1945, 1980, 2000, 2010, X['HouseYear'].max()]
    X['year_cat'] = pd.cut(X['HouseYear'], bins=bins, labels=False)
    
    X['year_cat'].fillna(-1, inplace=True)
    return X

In [ ]:
bins = [0, 3, 5, 9, 15, train_df['Floor'].max()]
pd.cut(train_df['Floor'], bins=bins, labels=False)

In [ ]:
bins = [0, 3, 5, 9, 15, train_df['Floor'].max()]
pd.cut(train_df['Floor'], bins=bins)

In [ ]:
train_df = year_to_cat(train_df)
train_df = floor_to_cat(train_df)
train_df.head()

In [ ]:
med_price_by_floor_year = train_df.groupby(['year_cat', 'floor_cat'], as_index=False).agg({'Price':'median'}).\
                                            rename(columns={'Price':'MedPriceByFloorYear'})
med_price_by_floor_year.head()

In [ ]:
train_df = train_df.merge(med_price_by_floor_year, on=['year_cat', 'floor_cat'], how='left')
train_df.head()

In [ ]:
class FeatureGenetator():
    """Генерация новых фич"""
    
    def __init__(self):
        self.DistrictId_counts = None
        self.binary_to_numbers = None
        self.med_price_by_district = None
        self.med_price_by_floor_year = None
        self.house_year_max = None
        self.floor_max = None
        
    def fit(self, X, y=None):
        
        X = X.copy()
        
        # Binary features
        self.binary_to_numbers = {'A': 0, 'B': 1}
        
        # DistrictID
        self.district_size = X['DistrictId'].value_counts().reset_index() \
                               .rename(columns={'index':'DistrictId', 'DistrictId':'DistrictSize'})
                
        # Target encoding
        ## District, Rooms
        df = X.copy()
        
        if y is not None:
            df['Price'] = y.values
            
            self.med_price_by_district = df.groupby(['DistrictId', 'Rooms'], as_index=False).agg({'Price':'median'})\
                                            .rename(columns={'Price':'MedPriceByDistrict'})
            
            self.med_price_by_district_median = self.med_price_by_district['MedPriceByDistrict'].median()
            
        ## floor, year
        if y is not None:
            self.floor_max = df['Floor'].max()
            self.house_year_max = df['HouseYear'].max()
            df['Price'] = y.values
            df = self.floor_to_cat(df)
            df = self.year_to_cat(df)
            self.med_price_by_floor_year = df.groupby(['year_cat', 'floor_cat'], as_index=False).agg({'Price':'median'}).\
                                            rename(columns={'Price':'MedPriceByFloorYear'})
            self.med_price_by_floor_year_median = self.med_price_by_floor_year['MedPriceByFloorYear'].median()
        

        
    def transform(self, X):
        
        # Binary features
        X['Ecology_2'] = X['Ecology_2'].map(self.binary_to_numbers)  # self.binary_to_numbers = {'A': 0, 'B': 1}
        X['Ecology_3'] = X['Ecology_3'].map(self.binary_to_numbers)
        X['Shops_2'] = X['Shops_2'].map(self.binary_to_numbers)
        
        # DistrictId, IsDistrictLarge
        X = X.merge(self.district_size, on='DistrictId', how='left')
        
        X['new_district'] = 0
        X.loc[X['DistrictSize'].isna(), 'new_district'] = 1

        X['First_Floor'] = 0
        X.loc[X['Floor'] == 1, 'First_Floor'] = 1
        X['Last_Floor'] = 0
        X.loc[X['Floor'] == X['HouseFloor'], 'Last_Floor'] = 1
        
        X['DistrictSize'].fillna(5, inplace=True)
        
        X['IsDistrictLarge'] = (X['DistrictSize'] > 100).astype(int)
        
        # More categorical features
        X = self.floor_to_cat(X)  # + столбец floor_cat
        X = self.year_to_cat(X)   # + столбец year_cat
        
        # Target encoding
        if self.med_price_by_district is not None:
            X = X.merge(self.med_price_by_district, on=['DistrictId', 'Rooms'], how='left')
            X.fillna(self.med_price_by_district_median, inplace=True)
            
        if self.med_price_by_floor_year is not None:
            X = X.merge(self.med_price_by_floor_year, on=['year_cat', 'floor_cat'], how='left')
            X.fillna(self.med_price_by_floor_year_median, inplace=True)
        
        return X
    
    def floor_to_cat(self, X):
        bins = [0, 3, 5, 9, 15, self.floor_max]
        X['floor_cat'] = pd.cut(X['Floor'], bins=bins, labels=False)

        X['floor_cat'].fillna(-1, inplace=True) 
        return X
     
    def year_to_cat(self, X):
        bins = [0, 1941, 1945, 1980, 2000, 2010, self.house_year_max]
        X['year_cat'] = pd.cut(X['HouseYear'], bins=bins, labels=False)

        X['year_cat'].fillna(-1, inplace=True)
        return X           
    

5. Отбор признаков

In [ ]:
train_df.columns.tolist()

In [ ]:
feature_names = ['Rooms', 'Square', 'LifeSquare', 'KitchenSquare', 'Floor', 'HouseFloor', 'HouseYear',
                 'Ecology_1', 'Ecology_2', 'Ecology_3', 'Social_1', 'Social_2', 'Social_3',
                 'Helthcare_2', 'Shops_1']

new_feature_names = ['Rooms_outlier', 'HouseFloor_outlier', 'HouseYear_outlier', 'LifeSquare_nan', 'DistrictSize',
                     'new_district', 'IsDistrictLarge',  'MedPriceByFloorYear', 'First_Floor', 'Last_Floor']

target_name = 'Price'

Normal distribution of the target variable

In [ ]:
from scipy import stats

sns.distplot(train_df['Price'], fit=norm)

mu, sigma = norm.fit(train_df['Price'])

print(f'mu = {mu:.2f} and sigma = {sigma:.2f}')

plt.legend(
    [f'Normal dist. ($\mu=$ {mu:.2f} and $\sigma=$ {sigma:.2f} )'], loc='best')
plt.ylabel('Frequency')
plt.title('Price distribution')

# QQ-plot
fig = plt.figure()
res = stats.probplot(train_df['Price'], plot=plt)
plt.show()

Data Correlation

In [ ]:
corrmat = train_df.loc[:, train_df.columns != 'Id'].corr()
plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=0.9, square=True)

In [ ]:
corrmat = train_df.loc[:, train_df.columns != 'Id'].corrwith(
    train_df['Price']).abs().sort_values(ascending=False)[1:]
plt.bar(corrmat.index, corrmat.values)
plt.title('Correlation to Price')
plt.xticks(rotation='vertical')
plt.show()

6. Разбиение на train и test

In [ ]:
train_df = pd.read_csv(TRAIN_DATASET_PATH)
test_df = pd.read_csv(TEST_DATASET_PATH)

X = train_df.drop(columns=target_name)
y = train_df[target_name]

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33, shuffle=True, random_state=21)

In [ ]:
preprocessor = DataPreprocessing()
preprocessor.fit(X_train)

X_train = preprocessor.transform(X_train)
X_valid = preprocessor.transform(X_valid)
test_df = preprocessor.transform(test_df)

X_train.shape, X_valid.shape, test_df.shape

In [ ]:
features_gen = FeatureGenetator()
features_gen.fit(X_train, y_train)

X_train = features_gen.transform(X_train)
X_valid = features_gen.transform(X_valid)
test_df = features_gen.transform(test_df)

X_train.shape, X_valid.shape, test_df.shape

In [ ]:
X_train = X_train[feature_names + new_feature_names]
X_valid = X_valid[feature_names + new_feature_names]
test_df = test_df[feature_names + new_feature_names]

In [ ]:
X_train.isna().sum().sum(), X_valid.isna().sum().sum(), test_df.isna().sum().sum()

In [ ]:
7. Построение модели

Подбор оптимальной модели

In [ ]:
#from sklearn.model_selection import RandomizedSearchCV
#from pprint import pprint

## Number of trees in random forest
#n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
## Number of features to consider at every split
#max_features = ['auto', 'sqrt']
## Maximum number of levels in tree
#max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
#max_depth.append(None)
## Minimum number of samples required to split a node
#min_samples_split = [2, 5, 10]
## Minimum number of samples required at each leaf node
#min_samples_leaf = [1, 2, 4]
## Method of selecting samples for training each tree
#bootstrap = [True, False]
## Create the random grid
#random_grid = {'n_estimators': [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)],
#                'importance_type':['weight', 'gain', 'cover', 'total_gain', 'total_cover'],
#                'booster':['gbtree', 'gblinear', 'dart'],
#                'max_depth': [3, 4, 5, 6, 7, 8, 9, 10],
#                'tree_method':['auto', 'exact', 'approx', 'hist', 'gpu_hist'],
#                'subsample': [0.9, 1.0],
#                'colsample_bytree': [0.9, 1.0]}
           
#pprint(random_grid)

### Use the random grid to search for best hyperparameters
## First create the base model to tune
#rf = XGBRegressor(random_state=21)
## Random search of parameters, using 3 fold cross validation, 
## search across 100 different combinations, and use all available cores
#rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 3, verbose=2, random_state=21, n_jobs = -1)
## Fit the random search model
#rf_random.fit(X_train, y_train)
#rf_random.best_params_

#XGBRegressor(random_state=21,
#               tree_method='hist',
#               subsample=1.0,
#               n_estimators=600,
#               max_depth=3,
#               importance_type='total_cover',
#               colsample_bytree=1.0,
#               booster='dart')

In [ ]:
#from sklearn.model_selection import RandomizedSearchCV
#from pprint import pprint

## Number of trees in random forest
#n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
## Number of features to consider at every split
#max_features = ['auto', 'sqrt']
## Maximum number of levels in tree
#max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
#max_depth.append(None)
## Minimum number of samples required to split a node
#min_samples_split = [2, 5, 10]
## Minimum number of samples required at each leaf node
#min_samples_leaf = [1, 2, 4]
## Method of selecting samples for training each tree
#bootstrap = [True, False]
## Create the random grid
#random_grid = {'n_estimators': n_estimators,
#               'max_features': max_features,
#               'max_depth': max_depth,
#               'min_samples_split': min_samples_split,
#               'min_samples_leaf': min_samples_leaf,
#               'bootstrap': bootstrap}
#pprint(random_grid)


### Use the random grid to search for best hyperparameters
## First create the base model to tune
#rf = RandomForestRegressor(random_state=21, criterion='mse')
## Random search of parameters, using 3 fold cross validation, 
## search across 100 different combinations, and use all available cores
#rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 3, verbose=2, random_state=21, n_jobs = -1)
## Fit the random search model
#rf_random.fit(X_train, y_train)
#rf_random.best_params_

#(random_state=21,
# criterion='mse',
# n_estimators=1800,
# min_samples_split=2,
# min_samples_leaf=2,
# max_features='sqrt',
# max_depth=80,
# bootstrap=False)

In [ ]:
#from pprint import pprint
#from lightgbm import LGBMRegressor

## Create the random grid
#n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
#max_bin  = [int(x) for x in np.linspace(10, 110, num = 11)]
#random_grid = {
#    'num_leaves': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
#    'max_depth': [1, 2, 3, 4, 5],
#    'max_bin': max_bin,
#    'n_estimators': np.arange(1000, 7000, 1000),
#    'learning_rate': np.arange(0.01, 0.05, 0.01)
#}
#pprint(random_grid)

#### Use the random grid to search for best hyperparameters
### First create the base model to tune
#lgbm = LGBMRegressor(random_state=21)
### Random search of parameters, using 3 fold cross validation, 
### search across 100 different combinations, and use all available cores
#lgbm_random = RandomizedSearchCV(estimator = lgbm, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=21, n_jobs = -1)
## Fit the random search model
#lgbm_random.fit(X_train, y_train)

#print(lgbm_random.best_params_)


####LGBMRegressor(random_state=21, 
####              num_leaves=10,
####              n_estimators=1000,
####              max_depth=5,
####              max_bin=30,
####              learning_rate= 0.03)

####rf_model = LGBMRegressor(random_state=21, 
####                         num_leaves=5,
####                         n_estimators=3000,
####                         max_depth=5,
####                         max_bin=80,
####                         learning_rate= 0.03)

In [ ]:
#from sklearn.model_selection import RandomizedSearchCV
#from pprint import pprint
## Number of trees in random forest
#n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
## Number of features to consider at every split
#max_features = ['auto', 'sqrt', 'log2']
## Maximum number of levels in tree
#max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
#max_depth.append(None)
## Minimum number of samples required to split a node
#min_samples_split = [2, 5, 8, 10, 12]
## Minimum number of samples required at each leaf node
#min_samples_leaf = [1, 2, 3, 4, 5]
## The function to measure the quality of a split
#criterion = ['mse']
## Create the random grid
#random_grid = {'criterion': criterion,
#               'max_depth': max_depth,
#               'max_features': max_features,
#               'min_samples_leaf': min_samples_leaf,
#               'min_samples_split': min_samples_split,
#               'n_estimators': np.arange(1000, 7000, 1000),}
#pprint(random_grid)


### Use the random grid to search for best hyperparameters
## First create the base model to tune
#gb = GradientBoostingRegressor(random_state=21)
## Random search of parameters, using 3 fold cross validation, 
## search across 100 different combinations, and use all available cores
#gb_random = RandomizedSearchCV(estimator = gb, param_distributions = random_grid, n_iter = 10, cv = 3, verbose=2, random_state=21, n_jobs = -1)
## Fit the random search model
#gb_random.fit(X_train, y_train)
#gb_random.best_params_


#{'n_estimators': 3000,
# 'min_samples_split': 12,
# 'min_samples_leaf': 5,
# 'max_features': 'sqrt',
# 'max_depth': 40,
# 'criterion': 'mse'}

In [ ]:
####r = RandomForestRegressor(random_state=21,
####                                 criterion='mse',
####                                 n_estimators=1800,
####                                 min_samples_split=2,
####                                 min_samples_leaf=2,
####                                 max_features='sqrt',
####                                 max_depth=80,
####                                 bootstrap=False)

####gb = GradientBoostingRegressor(random_state=21,
####                                    n_estimators=3000,
####                                    min_samples_split=12,
####                                    min_samples_leaf=5,
####                                    max_features='sqrt',
####                                    max_depth=40,
####                                    criterion='mse')

####lgb = LGBMRegressor(random_state=21, 
####              num_leaves=10,
####              n_estimators=1000,
####              max_depth=5,
####              max_bin=30,
####              learning_rate= 0.03)

####xgb = XGBRegressor(random_state=21,
####                       tree_method='hist',
####                       subsample=1.0,
####                       n_estimators=600,
####                       max_depth=3,
####                       importance_type='total_cover',
####                       colsample_bytree=1.0,
####                       booster='dart')
####er_model = VotingRegressor([('r', r), ('gb', gb), ('lgb', lgb), ('xgb', xgb)])
####er_model.fit(X_train, y_train)

####y_train_preds = er_model.predict(X_train)
####y_test_preds = er_model.predict(X_valid)

####evaluate_preds(y_train, y_train_preds, y_valid, y_test_preds)
####from sklearn.ensemble import StackingRegressor, VotingRegressor, BaggingRegressor, GradientBoostingRegressor

####r = RandomForestRegressor(random_state=21,
####                                 criterion='mse',
####                                 n_estimators=1800,
####                                 min_samples_split=2,
####                                 min_samples_leaf=2,
####                                 max_features='sqrt',
####                                 max_depth=80,
####                                 bootstrap=False)

####gb = GradientBoostingRegressor(random_state=21,
####                                    n_estimators=3000,
####                                    min_samples_split=12,
####                                    min_samples_leaf=5,
####                                    max_features='sqrt',
####                                    max_depth=40,
####                                    criterion='mse')

####lgb = LGBMRegressor(random_state=21, 
####              num_leaves=10,
####              n_estimators=1000,
####              max_depth=5,
####              max_bin=30,
####              learning_rate= 0.03)

####xgb = XGBRegressor(random_state=21,
####                       tree_method='hist',
####                       subsample=1.0,
####                       n_estimators=600,
####                       max_depth=3,
####                       importance_type='total_cover',
####                       colsample_bytree=1.0,
####                       booster='dart')

####stack = StackingRegressor([('r',r), ('gb',gb), ('xgb',xgb), ('lbg', lgb))
####stack.fit(X_train, y_train)

####y_train_preds = er_model.predict(X_train)
####y_test_preds = er_model.predict(X_valid)

####evaluate_preds(y_train, y_train_preds, y_valid, y_test_preds)
####bg_model = BaggingRegressor(base_estimator=rf_model,  n_estimators=8, random_state=0)
####bg_model.fit(X_train, y_train)

####y_train_preds = bg_model.predict(X_train)
####y_test_preds = bg_model.predict(X_valid)

####evaluate_preds(y_train, y_train_preds, y_valid, y_test_preds)

In [ ]:
#rf_model = RandomForestRegressor(random_state=21,
#                                 criterion='mse',
#                                 n_estimators=4000,
#                                 min_samples_split=4,
#                                 min_samples_leaf=1,
#                                 max_features='sqrt',
#                                 max_depth=100,
#                                 bootstrap=False)
#rf_model.fit(X_train, y_train)

**Обучение

In [ ]:
lgbm_random = LGBMRegressor(random_state=21, 
              n_estimators=4000,
              max_depth=8,
              max_bin=150,
              num_leaves=10,
              learning_rate=0.01)

rf_model = RandomForestRegressor(random_state=21,
                                 criterion='mse',
                                 n_estimators=4000,
                                 min_samples_split=4,
                                 min_samples_leaf=1,
                                 max_features='sqrt',
                                 max_depth=100,
                                 bootstrap=False)

stack = StackingRegressor([('lgb',lgbm_random), ('rf',rf_model)])
stack.fit(X_train, y_train)

**Оценка модели

In [ ]:
y_train_preds = stack.predict(X_train)
y_test_preds = stack.predict(X_valid)

evaluate_preds(y_train, y_train_preds, y_valid, y_test_preds)

Кросс-валидация

In [ ]:
cv_score = cross_val_score(stack, X_train, y_train, scoring='r2', cv=KFold(n_splits=3, shuffle=True, random_state=21))
cv_score

In [ ]:
cv_score.mean()

Важность признаков

In [ ]:
#feature_importances = pd.DataFrame(zip(X_train.columns, stack.feature_importances_), 
#                                    columns=['feature_name', 'importance'])

#feature_importances.sort_values(by='importance', ascending=False)

8. Прогнозирование на тестовом датасете
Выполнить для тестового датасета те же этапы обработки и постронияния признаков
Не потерять и не перемешать индексы от примеров при построении прогнозов
Прогнозы должны быть для все примеров из тестового датасета (для всех строк)

In [ ]:
test_df.shape

In [ ]:
test_df

In [ ]:
submit = pd.read_csv('/kaggle/input/real-estate-price-prediction-moscow/sample_submission.csv')
submit.head()

In [ ]:
predictions = stack.predict(test_df)
predictions

In [ ]:
submit['Price'] = predictions
submit.head()

In [ ]:
submit.to_csv('rf_submit.csv', index=False)